<a href="https://colab.research.google.com/github/mehdihosseinimoghadam/Signal-Processing/blob/main/WaveNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import pathlib
import random
import itertools
from tqdm import tqdm

from IPython import display
from dataclasses import dataclass

import torch
import torch.nn.functional as F
from torch import distributions
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

import torchaudio
from torchaudio.transforms import MelSpectrogram

import librosa
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [2]:
batch_size = 1
in_channel = 1
num_samples = 10

input = torch.arange(num_samples).reshape(batch_size, in_channel, num_samples).float()
print(f'input: {input}')

# Let's omit bias for simplicy
conv_1d = nn.Conv1d(in_channel, in_channel, 3, bias=False)
print(f'weights: {conv_1d.weight}')

# Turn weights into `ones`
nn.init.ones_(conv_1d.weight)

input: tensor([[[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]])
weights: Parameter containing:
tensor([[[-0.1340,  0.1893,  0.2076]]], requires_grad=True)


Parameter containing:
tensor([[[1., 1., 1.]]], requires_grad=True)

In [3]:
print(f'Input: {input.squeeze()}')
print(f'After Conv1D: {conv_1d(input).squeeze()}')

Input: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
After Conv1D: tensor([ 3.,  6.,  9., 12., 15., 18., 21., 24.], grad_fn=<SqueezeBackward0>)


In [18]:
class CausalConv1d(nn.Conv1d):
    """
    Casual Conv1d
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int,
        dilation: int = 1,
        bias: bool = False
    ):
        super().__init__(
            in_channels,
            out_channels,
            kernel_size,
            dilation=dilation,
            bias=bias
        )

        padding_size = (kernel_size - 1) * dilation
        self.zero_padding = nn.ConstantPad1d(
            padding=(padding_size, 0),
            value=0.0
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        print(input)
        padded_input = self.zero_padding(input)
        print(padded_input)
        output = super().forward(padded_input)
        print(output)
        return output

In [19]:
causal_block = CausalConv1d(in_channels=1, out_channels=1, kernel_size=3, dilation=3)
nn.init.ones_(causal_block.weight)

Parameter containing:
tensor([[[1., 1., 1.]]], requires_grad=True)

In [20]:
print(f'Input: {input.squeeze()}')
print(f'After Conv1D: {causal_block(input).squeeze()}')

Input: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
tensor([[[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]])
tensor([[[0., 0., 0., 0., 0., 0., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]])
tensor([[[ 0.,  1.,  2.,  3.,  5.,  7.,  9., 12., 15., 18.]]],
       grad_fn=<SqueezeBackward1>)
After Conv1D: tensor([ 0.,  1.,  2.,  3.,  5.,  7.,  9., 12., 15., 18.],
       grad_fn=<SqueezeBackward0>)
